In [1]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('./weathercache/.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": 12.9719,
	"longitude": 77.5937,
    "timezone": "auto",
    # "forecast_days": 1,
    "current_weather": True,
	"hourly": ["temperature_2m", "relative_humidity_2m", "dew_point_2m", "apparent_temperature", "rain", "precipitation", "showers", "precipitation_probability", "snowfall", "snow_depth"]
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()}{response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
hourly_dew_point_2m = hourly.Variables(2).ValuesAsNumpy()
hourly_apparent_temperature = hourly.Variables(3).ValuesAsNumpy()
hourly_rain = hourly.Variables(4).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(5).ValuesAsNumpy()
hourly_showers = hourly.Variables(6).ValuesAsNumpy()
hourly_precipitation_probability = hourly.Variables(7).ValuesAsNumpy()
hourly_snowfall = hourly.Variables(8).ValuesAsNumpy()
hourly_snow_depth = hourly.Variables(9).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_data["dew_point_2m"] = hourly_dew_point_2m
hourly_data["apparent_temperature"] = hourly_apparent_temperature
hourly_data["rain"] = hourly_rain
hourly_data["precipitation"] = hourly_precipitation
hourly_data["showers"] = hourly_showers
hourly_data["precipitation_probability"] = hourly_precipitation_probability
hourly_data["snowfall"] = hourly_snowfall
hourly_data["snow_depth"] = hourly_snow_depth

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)

Coordinates 13.0°N 77.625°E
Elevation 918.0 m asl
Timezone b'Asia/Kolkata'b'GMT+5:30'
Timezone difference to GMT+0 19800 s
                         date  temperature_2m  relative_humidity_2m  \
0   2025-04-07 18:30:00+00:00       24.546000                  59.0   
1   2025-04-07 19:30:00+00:00       23.646000                  64.0   
2   2025-04-07 20:30:00+00:00       22.896000                  70.0   
3   2025-04-07 21:30:00+00:00       22.246000                  76.0   
4   2025-04-07 22:30:00+00:00       21.746000                  82.0   
..                        ...             ...                   ...   
163 2025-04-14 13:30:00+00:00       26.496000                  65.0   
164 2025-04-14 14:30:00+00:00       25.445999                  71.0   
165 2025-04-14 15:30:00+00:00       24.846001                  72.0   
166 2025-04-14 16:30:00+00:00       24.496000                  70.0   
167 2025-04-14 17:30:00+00:00       24.246000                  69.0   

     dew_point_2m  appar

In [12]:
hourly

## Import Necessary Modules 

In [2]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from dotenv import load_dotenv
import os
from groq import Groq
from langchain_groq import ChatGroq
import ipywidgets as widgets
from IPython.display import display, HTML
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import chain
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.chains import LLMChain
from langchain_core.documents import Document
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_distances
from langchain_chroma import Chroma
from uuid import uuid4
from langchain_community.vectorstores.utils import filter_complex_metadata
from langchain.retrievers import BM25Retriever, EnsembleRetriever

## Populating the Vector Database
Check populate_vector_bd.ipynb file

## Load Vector DB or Create Retriever

In [ ]:
persist_directory="./Dataset/chroma_db_langchain"
# Embeddings
embedder = HuggingFaceEmbeddings(model_name="BAAI/llm-embedder")

In [ ]:
vectordb = Chroma(persist_directory=persist_directory, 
                collection_name="Adavance_RAG_Test",
                embedding_function=embedder)